In [1]:
import os
import string
import re
from pathlib import Path

In [5]:
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)
data_dir = current_dir.joinpath('data')
data_dir.mkdir(parents=True, exist_ok=True)
external_data_dir = current_dir.parent.parent.parent.joinpath('data')
imdb_dir = external_data_dir.joinpath(r'external/imdb/aclImdb')




print(current_dir)
print(results_dir)
print(data_dir)
print(imdb_dir)

/home/jovyan/dsc650/dsc650/assignments/assignment10
/home/jovyan/dsc650/dsc650/assignments/assignment10/results
/home/jovyan/dsc650/dsc650/assignments/assignment10/data
/home/jovyan/dsc650/data/external/imdb/aclImdb


#### Assignment 10.1.a  

Create a tokenize function that splits a sentence into words. Ensure that your tokenizer removes basic punctuation.

In [32]:
file_name = "sample.txt"
file_path = f"{data_dir}\{file_name}"
file_path

'c:\\Users\\saman\\git_repos\\dsc650\\dsc650\\assignments\\assignment10\\data\\sample.txt'

In [ ]:
def tokenize(sentence):
    tokens = []
    words = sentence.split()
    # tokenize the sentence
    for word in words:
        # allowing only alphabets
        word = re.sub("[^a-zA-Z]", "", word)
        tokens.append(word)
    return tokens

In [ ]:
def call_tokenize():
	with open(f'{file_path}','r') as f:
		for line in f:
			line = line.lower()
			# print(line, end = 'XX')
			return tokenize(line)

call_tokenize()

#### Assignment 10.1.b

Implement an `ngram` function that splits tokens into N-grams. 

In [ ]:
def ngram(tokens, n):
    # ngrams = []
    # Create ngrams
    # return ngrams
    return print(list(zip(*[tokens[i:] for i in range(n)])))

In [ ]:
def call_ngram():
	with open(f'{file_path}','r') as f:
		for line in f:
			line = line.lower()
			# print(line, end = 'XX')
			return ngram(tokenize(line),3)

call_ngram()

#### Assignment 10.1.c

Implement an one_hot_encode function to create a vector from a numerical vector from a list of tokens.

In [ ]:
def one_hot_encode(tokens, num_words):
    token_index = {}
    results = ''
    return results

#### 10.2

Using listings 6.16, 6.17, and 6.18 in Deep Learning with Python as a guide, train a sequential model with embeddings on the IMDB data found in data/external/imdb/. Produce the model performance metrics and training and validation accuracy curves within the Jupyter notebook.

In [6]:
# Processing the labels of the raw IMDB data
train_dir = imdb_dir.joinpath('train')
labels = []
texts = []

for label_type in ['neg','pos']:
	dir_name = train_dir.joinpath(label_type) 
	for fname in os.listdir(dir_name):
		if fname[-4:] == '.txt':
			f = open(dir_name.joinpath(fname))
			texts.append(f.read())
			f.close()
			if label_type == 'neg':
				labels.append(0)
			else:
				labels.append(1)

In [ ]:
# Tokenizing the text of the raw IMDB data

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np